# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import utils
import pathing

import random
import time

import networkx
import osmnx as ox
import numpy as np
import pandas as pd


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Methods

In [5]:
def fix_lat_lon(benchmark_times):
    new_benchmark_times = {}

    for ((start_lat, start_lon), (end_lat, end_lon)), benchmark in benchmark_times.items():
        new_benchmark_times[((start_lon, start_lat), (end_lon, end_lat))] = benchmark

    return new_benchmark_times

In [6]:

# times from Google Maps (Sunday, start at 02:00)
benchmark_times = {
    ((59.9369806, 10.7343661), (59.6800045, 10.6572104)): (35, 45),
    ((59.93596, 10.73244), (59.92629, 10.77570)): (4, 9),
    ((59.90864, 10.73921), (59.93037, 10.77273)): (9, 16),
    ((59.92727, 10.73174), (59.86305, 10.66617)): (50, 60),
    ((59.82052, 10.47168), (59.95577, 11.04773)): (40, 50),
    ((60.00352, 10.76216), (59.83905, 10.80742)): (28, 28),

    ((60.31262, 11.14674), (60.11964, 11.47577)): (40, 40),
    ((60.11964, 11.47577), (60.13423, 11.166858)): (22, 26),
    ((60.13423, 11.166858), (59.879044, 11.561687)): (50, 50),
    ((59.879044, 11.561687), (59.932076, 10.987775)): (40, 40),
    ((59.932076, 10.987775), (60.042152, 10.880784)): (20, 22),
    ((60.042152, 10.880784), (59.930733, 10.831094)): (20, 20),
    ((59.930733, 10.831094), (59.917, 10.758744)): (9, 14),
    ((59.917, 10.758744), (59.93917, 10.741926)): (7, 14),
    ((59.93917, 10.741926), (59.71566, 10.853331)): (35, 35),
    ((59.71566, 10.853331), (59.659687, 10.725881)): (18, 18),
    ((59.659687, 10.725881), (59.833073, 10.806946)): (22, 22),
    ((59.833073, 10.806946), (59.830055, 10.439887)): (30, 40),
    ((59.830055, 10.439887), (59.89813, 10.509723)): (14, 14),
    ((59.89813, 10.509723), (59.939663, 10.687367)): (16, 20),
    ((59.939663, 10.687367), (59.893173, 10.806364)): (12, 16),
    ((59.893173, 10.806364), (59.960304, 10.884091)): (12, 18),
    ((59.960304, 10.884091), (59.997875, 11.03928)): (20, 20),
    ((59.997875, 11.03928), (59.917873, 10.585751)): (28, 35),
    ((59.917873, 10.585751), (60.31262, 11.14674)): (55, 65),
}

benchmark_times = fix_lat_lon(benchmark_times)



In [38]:
# times from Google Maps (Sunday, start at 02:00)
test_benchmark_times = {
    ((59.92289773858113, 10.679167138180098), (59.920308050739855, 10.770082007256002)): (10, 16),
    ((59.95345016032127, 10.760869824639984), (59.88350174573856, 10.77162290351496)): (20,24),
    ((59.857600021161346, 10.43132928126165), (60.05149938976296, 10.83919667864857)): (40, 50),
    ((60.053190496575645, 10.854342819294414), (59.9274978707624, 11.185305926264945)): (30, 30),
    ((59.809771557951564, 10.760794771211929), (60.148013543553006, 11.296246670387003)): (44, 44),
}

test_benchmark_times = fix_lat_lon(test_benchmark_times)


In [7]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


In [33]:
def calculate_error(od: pathing.OriginDestination, benchmark_times):
    calculated_times = {}
    
    # Assuming utils and get_node functions are defined and work as intended
    for ((start_lon, start_lat), (end_lon, end_lat)), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat)

        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time

    # Calculate MSE
    errors = []

    for path, acceptable_range in benchmark_times.items():
        calculated_time = calculated_times[path]
        lower_bound, upper_bound = acceptable_range

        # Check if the calculated time falls within the acceptable range
        if lower_bound <= calculated_time <= upper_bound:
            error = 0
        else:
            # If outside the range, find the smallest absolute difference to the bounds
            error = min(abs(calculated_time - lower_bound), abs(calculated_time - upper_bound))
        
        errors.append(error)

    mse = sum(error ** 2 for error in errors) / len(errors)
    
    return calculated_times


In [9]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


# Main

In [17]:
default_intersection_penalty = 2
traffic_signal_penalty = 5
use_ambulance_speeds=False
road_type_factors= {
        "motorway": 0.9704026402720763,
        "trunk": 0.9872728848094453,
        "primary": 0.9434615680765118,
        "secondary": 0.49062545330175955,
        "tertiary": 0.9327873614166583,
        "unclassified": 0.8502498894729571,
        "residential": 0.7543590170147566,
        "living_street": 0.8078248863334735,
    }

In [18]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

In [34]:
od.get_graph()

In [39]:
od.set_graph_weights_v2(default_intersection_penalty, traffic_signal_penalty, road_type_factors, use_ambulance_speeds)
calculated_times = calculate_error(od, test_benchmark_times)

od.set_graph_weights_v2(default_intersection_penalty, traffic_signal_penalty, road_type_factors, use_ambulance_speeds=True)
calculated_ambulance_times = calculate_error(od, test_benchmark_times)

In [41]:
def compare_times(od, benchmark_times, normal_times, ambulance_times):
    # Calculate computed car times just like before, if not already calculated
    computed_car_times = {}
    for ((start_lon, start_lat), (end_lon, end_lat)) in benchmark_times.keys():
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat)
        
        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)
        
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))
        computed_car_times[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time
    
    # Now, compare benchmark, normal_times (computed car times), and ambulance_times
    for path, benchmark_range in benchmark_times.items():
        bench_low, bench_high = benchmark_range
        try:
            car_time = normal_times[path]
        except KeyError:
            car_time = 0
        ambulance_time = ambulance_times.get(path, 0)
        
        print(f"Path {path}:")
        print(f"  Benchmark Time: {bench_low} to {bench_high} minutes")
        print(f"  Computed Car Time: {car_time:.2f} minutes")
        print(f"  Computed Ambulance Time: {ambulance_time:.2f} minutes\n")

compare_times(od, test_benchmark_times, calculated_times, calculated_ambulance_times)


Path ((10.679167138180098, 59.92289773858113), (10.770082007256002, 59.920308050739855)):
  Benchmark Time: 10 to 16 minutes
  Computed Car Time: 9.84 minutes
  Computed Ambulance Time: 8.35 minutes

Path ((10.760869824639984, 59.95345016032127), (10.77162290351496, 59.88350174573856)):
  Benchmark Time: 20 to 24 minutes
  Computed Car Time: 12.14 minutes
  Computed Ambulance Time: 9.37 minutes

Path ((10.43132928126165, 59.857600021161346), (10.83919667864857, 60.05149938976296)):
  Benchmark Time: 40 to 50 minutes
  Computed Car Time: 52.75 minutes
  Computed Ambulance Time: 42.56 minutes

Path ((10.854342819294414, 60.053190496575645), (11.185305926264945, 59.9274978707624)):
  Benchmark Time: 30 to 30 minutes
  Computed Car Time: 27.55 minutes
  Computed Ambulance Time: 20.55 minutes

Path ((10.760794771211929, 59.809771557951564), (11.296246670387003, 60.148013543553006)):
  Benchmark Time: 44 to 44 minutes
  Computed Car Time: 54.95 minutes
  Computed Ambulance Time: 44.34 minute